<a href="https://colab.research.google.com/github/Vyoma-garg/Natural-Language-Processing/blob/main/LSTM_with_only_last_hidden_state.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn  #all the NN network, CNN, Modules, loss functions, activation fns
import torch.optim as optim  #SGD, ADAM
import torch.nn.functional as F  #activation fn, 
from torch.utils.data import DataLoader  #easier data managemnet, creates mini batches of the data
import torchvision.datasets as datasets   #standard datasets MNISt ,etc
import torchvision.transforms as transforms  # transformations 

Create LSTM with only last hidden state

In [27]:
class LSTM(nn.Module): #inherit from nn.Module class
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM,self).__init__()   
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm= nn.LSTM(input_size, hidden_size, num_layers, batch_first=True )
        #returns batch_size x seq x features --> 64x28x28

        self.fc = nn.Linear(hidden_size,num_classes)  #for each hidden state

    def forward(self, x): 
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
         # init as num of layers, size: how many min batches we need to send in 
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
         #forward 
        out,_ = self.lstm(x, (h0,c0))
        #out=out.reshape(out.shape[0],-1)  #28x256
        #for only last hidden state
        out = self.fc(out[:,-1,:])  #all batches at a same time, last hidden state,all features
        return out



Checking 

In [4]:
#model = CNN()
#x = torch.randn(64,1,28,28)  #randomly generated data, batch size=4, number of examples used simultaneously
#print(x)
#print('\n',model(x).shape,'\n')  #64x10   
#print(model(x)) 

Setting device

In [5]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

Hyperparameters

In [28]:
input_size=28  #28 times seq, each seq have 28 features
sequence_length=28  #each row at time stamp
num_layers=2
hidden_size=256
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=1

Loading dataset

In [17]:
train_dataset = datasets.MNIST(root = '/content', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = '/content', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

Initialize network

In [29]:
model=LSTM(input_size, hidden_size, num_layers, num_classes).to(device)

Loss and Optimizer

In [30]:
criterion= nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=learning_rate)

Train Network

In [31]:
for epoch in range(num_epochs):
  for batch_idx,(data,targets) in enumerate(train_loader):  #enumerate helps in getting the batch index of the data 
     #Get data to cuda
      data = data.to(device).squeeze(1)  #([64,1,28,28])  --> ([64,28,28])
      targets = targets.to(device)

      #print(data.shape)  #([64,1,28,28])  num of examples, number of channels(black white img), height width of each image
                        
      #get correct shape
      #data=data.reshape(data.shape[0],-1)  #flatten to single dim: ([64,784])
      #print(data.shape) 

      #forward
      scores=model(data)
      loss=criterion(scores, targets) 

      #backward
      optimizer.zero_grad() #
      loss.backward()

      #gradient descent or adam step
      optimizer.step() #update weights based on the gradients



Check accuracy

In [32]:
def check_accuracy(loader,model):
  if loader.dataset.train:
        print('Checking Accuracy on Training Data')
  else:
        print('Checking Accuracy on Test Data')
  num_correct=0
  num_samples=0
  model.eval()

  with torch.no_grad():  #while checking accuracy no need to calculate the gradients again
    for x,y in loader:
      x=x.to(device=device).squeeze(1)
      y=y.to(device=device)
      #x=x.reshape(x.shape[0],-1)

      scores=model(x)  #64x10 shape of the scores, which is max of those 10 digits
      values,predictions =scores.max(1)  #interested in index of the max value
      num_correct += (predictions==y).sum()
      num_samples += predictions.size(0)  #first dimension that is 64 x 10==64
    print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

  model.train()
  

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 57808/ 60000 with accuracy 96.35 

Checking Accuracy on Test Data
Got 9628/ 10000 with accuracy 96.28 

